In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.formula.api import ols

from sklearn.model_selection import train_test_split
import sklearn.linear_model
import sklearn.feature_selection
import sklearn.preprocessing
from scipy import stats
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score

from sklearn.linear_model import LinearRegression
from sklearn.feature_selection import f_regression 
from math import sqrt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import math
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest, RFE, f_regression, SequentialFeatureSelector

from sklearn.model_selection import train_test_split

## 1.0  Load the tips dataset from either pydataset or seaborn.

In [2]:
from pydataset import data

tips = data('tips')
tips.head()

,total_bill,tip,sex,smoker,day,time,size
1,16.99,1.01,Female,No,Sun,Dinner,2
2,10.34,1.66,Male,No,Sun,Dinner,3
3,21.01,3.50,Male,No,Sun,Dinner,3
4,23.68,3.31,Male,No,Sun,Dinner,2
5,24.59,3.61,Female,No,Sun,Dinner,4


In [3]:
tips[['day']].value_counts()

day 
Sat     87
Sun     76
Thur    62
Fri     19
dtype: int64

In [4]:
tips['sex'] = tips.sex.map({'Female': 1, 'Male': 0})
tips['smoker'] = tips.smoker.map({'Yes': 1, 'No': 0})
tips['time'] = tips.time.map({'Dinner': 1, 'Lunch': 0})
tips['day'] = tips.day.map({'Sat': 1, 'Sun': 1, 'Thur': 0, 'Fri':0})
    

## 1.1 Create a column named price_per_person. This should be the total bill divided by the party size.

In [6]:
tips['price_per_person'] = round(tips['total_bill'] / tips['size'],2)

In [7]:
tips

,total_bill,tip,sex,smoker,day,time,size,price_per_person
1,16.99,1.01,1,0,1,1,2,8.49
2,10.34,1.66,0,0,1,1,3,3.45
3,21.01,3.50,0,0,1,1,3,7.00
4,23.68,3.31,0,0,1,1,2,11.84
5,24.59,3.61,1,0,1,1,4,6.15
...,...,...,...,...,...,...,...,...
240,29.03,5.92,0,0,1,1,3,9.68
241,27.18,2.00,1,1,1,1,2,13.59
242,22.67,2.00,0,1,1,1,2,11.34
243,17.82,1.75,0,0,1,1,2,8.91


## 1.2 Before using any of the methods discussed in the lesson, which features do you think would be most important for predicting the tip amount?

I think the total bill, tip and price per person are most important for predicting the tip amount. 

## 1.3  Use select k best to select the top 2 features for predicting tip amount. What are they?

In [8]:
# split on tips data, 80/20 train test split. 
train, test = train_test_split(tips, train_size=0.78, random_state=1349)
# 70/30 train validate split
train, validate = train_test_split(train, train_size=0.7, random_state=1349)

In [9]:
train.columns

Index(['total_bill', 'tip', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person'],
      dtype='object')

In [10]:
# create X & y version of train, where y is a series with just the target variable and X are all the features. 

X_train = train[['total_bill', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person']]
y_train = train.tip

X_validate = validate[['total_bill', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person']]
y_validate = train.tip

X_test = test[['total_bill', 'sex', 'smoker', 'day', 'time', 'size',
       'price_per_person']]
y_test = test.tip

In [11]:
# Scale (Make the thing)
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler, (fit the thing)
scaler.fit(X_train)

# Use the scaler to transform train, validate, test (use the thing)
X_train_scaled = scaler.transform(X_train)
X_validate_scaled = scaler.transform(X_validate)
X_test_scaled = scaler.transform(X_test)

In [12]:
# Like our other sklearn objects...
kbest = SelectKBest(f_regression, k=3)
kbest.fit(X_train_scaled, y_train)

SelectKBest(k=3, score_func=<function f_regression at 0x7f8a3deb4d30>)

In [13]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)
X_train_scaled

,total_bill,sex,smoker,day,time,size,price_per_person
0,0.211353,1.0,0.0,0.0,0.0,0.2,0.212766
1,0.406787,0.0,0.0,0.0,1.0,0.2,0.480736
2,0.543779,0.0,0.0,1.0,1.0,0.4,0.391029
3,0.661081,0.0,1.0,1.0,1.0,0.2,0.830362
4,0.572476,0.0,0.0,1.0,1.0,0.6,0.271420
5,0.412442,0.0,0.0,0.0,0.0,0.2,0.488787
6,0.157520,1.0,1.0,1.0,1.0,0.2,0.139160
7,0.302472,1.0,1.0,1.0,1.0,0.2,0.338125
8,0.284667,0.0,0.0,0.0,0.0,0.2,0.313399
9,0.440092,1.0,0.0,0.0,0.0,0.6,0.180564


In [14]:
#X_train_scaled.rename(columns={0: "total_bill", 1: "sex", 2: "smoker", 3:"day", 4:"time", 5:"size", 6: "price_per_person"},inplace=True)

In [16]:
kbest_results = pd.DataFrame(dict(p=kbest.pvalues_, f=kbest.scores_), index=X_train_scaled.columns)
kbest_results

,p,f
total_bill,5.994078e-24,154.843650
sex,4.127594e-02,4.247954
smoker,7.651684e-01,0.089594
day,2.081084e-01,1.600286
time,1.961835e-01,1.687721
size,3.246965e-10,46.328161
price_per_person,4.293164e-06,23.022395


In [17]:
X_train_scaled.columns[kbest.get_support()]

Index(['total_bill', 'size', 'price_per_person'], dtype='object')

In [18]:
X_train.columns[kbest.get_support()]


Index(['total_bill', 'size', 'price_per_person'], dtype='object')

In [117]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[kbest.get_support()]
)
X_train_transformed.head()

,total_bill,size,price_per_person
140,13.16,2.0,6.58
92,22.49,2.0,11.24
240,29.03,3.0,9.68
180,34.63,2.0,17.32
45,30.40,4.0,7.60


In [118]:
X_train_transformed = pd.DataFrame(
    kbest.transform(X_train_scaled),
    index=X_train_scaled.index,
    columns=X_train.columns[kbest.get_support()]
)
X_train_transformed.head()

,total_bill,size,price_per_person
0,0.211353,0.2,0.212766
1,0.406787,0.2,0.480736
2,0.543779,0.4,0.391029
3,0.661081,0.2,0.830362
4,0.572476,0.6,0.271420


## 1.4 Use recursive feature elimination to select the top 2 features for tip amount. What are they? 

In [70]:
from sklearn.linear_model import LinearRegression

In [71]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [72]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train.columns)

,rfe_ranking
total_bill,2
sex,5
smoker,1
day,6
time,4
size,1
price_per_person,3


In [73]:
X_train.columns[rfe.get_support()]

Index(['smoker', 'size'], dtype='object')

In [74]:
X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index=X_train.index,
    columns=X_train.columns[rfe.support_]
)
X_train_transformed.head()

,smoker,size
140,0.0,2.0
92,0.0,2.0
240,0.0,3.0
180,1.0,2.0
45,0.0,4.0


In [120]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X_train_scaled, y_train)



X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index=X_train_scaled.index,
    columns=X_train_scaled.columns[rfe.support_]
)
X_train_transformed.head()

,total_bill,price_per_person
0,13.16,6.58
1,22.49,11.24
2,29.03,9.68
3,34.63,17.32
4,30.40,7.60


In [122]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train_scaled.columns)


,rfe_ranking
total_bill,1
sex,5
smoker,3
day,6
time,4
size,2
price_per_person,1


In [121]:
X_train.columns[rfe.get_support()]

Index(['total_bill', 'price_per_person'], dtype='object')

## 1.5 Why do you think select k best and recursive feature elimination might give different answers for the top features? Does this change as you change the number of features your are selecting?

## 2. Write a function named select_kbest that takes in the predictors (X), the target (y), and the number of features to select (k) and returns the names of the top k selected features based on the SelectKBest class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [22]:
def select_kbest(X, y, k): 
    kbest = sklearn.feature_selection.SelectKBest(sklearn.feature_selection.f_regression, k=k)

    # fit the object
    kbest.fit(X, y)
    
    # use the object (.get_support() is that array of booleans to filter the list of column names)
    return X.columns[kbest.get_support()].tolist()


In [23]:
select_kbest(X_train_scaled, y_train, 3)

['total_bill', 'size', 'price_per_person']

##  3. Write a function named rfe that takes in the predictors, the target, and the number of features to select. It should return the top k features based on the RFE class. Test your function with the tips dataset. You should see the same results as when you did the process manually.

In [24]:
def select_rfe(X, y, k):
    # make the thing
    lm = sklearn.linear_model.LinearRegression()
    rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=k)

    # Fit the thing
    rfe.fit(X, y)
    
    # use the thing
    features_to_use = X.columns[rfe.support_].tolist()
    
    # we need to send show_feature_rankings a trained/fit RFE object
    all_rankings = show_features_rankings(X, rfe)
    
    return features_to_use, all_rankings

## 4. Load the swiss dataset and use all the other features to predict Fertility. Find the top 3 features using both select k best and recursive feature elimination (use the functions you just built to help you out).

In [25]:
swiss = data('swiss')
swiss.head()

,Fertility,Agriculture,Examination,Education,Catholic,Infant.Mortality
Courtelary,80.2,17.0,15,12,9.96,22.2
Delemont,83.1,45.1,6,9,84.84,22.2
Franches-Mnt,92.5,39.7,5,5,93.40,20.2
Moutier,85.8,36.5,12,7,33.77,20.3
Neuveville,76.9,43.5,17,15,5.16,20.6


In [26]:
swiss.columns

Index(['Fertility', 'Agriculture', 'Examination', 'Education', 'Catholic',
       'Infant.Mortality'],
      dtype='object')

In [28]:
X_train = swiss[['Agriculture', 'Examination', 'Education', 'Catholic',
       'Infant.Mortality']]
y_train = swiss[['Fertility']]

In [30]:
# Scale (Make the thing)
scaler = sklearn.preprocessing.MinMaxScaler()

# Fit the scaler, (fit the thing)
scaler.fit(X_train)

# Use the scaler to transform train, validate, test (use the thing)
X_train_scaled = scaler.transform(X_train)


In [31]:
kbest = SelectKBest(f_regression, k=3)
kbest.fit(X_train_scaled, y_train)

SelectKBest(k=3, score_func=<function f_regression at 0x7f8a3deb4d30>)

In [33]:
X_train_scaled = pd.DataFrame(X_train_scaled, columns = X_train.columns)

In [34]:
X_train_scaled.columns[kbest.get_support()].tolist()

['Examination', 'Education', 'Catholic']

In [38]:
lm = sklearn.linear_model.LinearRegression()
rfe = sklearn.feature_selection.RFE(lm, n_features_to_select=2)

    # Fit the thing
rfe.fit(X_train_scaled, y_train)
    
    

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [40]:
model = LinearRegression()
rfe = RFE(model, n_features_to_select=2)
rfe.fit(X_train, y_train)

RFE(estimator=LinearRegression(), n_features_to_select=2)

In [41]:
X_train_transformed = pd.DataFrame(
    rfe.transform(X_train),
    index=X_train_scaled.index,
    columns=X_train_scaled.columns[rfe.support_]
)
X_train_transformed.head()

,Education,Infant.Mortality
0,12.0,22.2
1,9.0,22.2
2,5.0,20.2
3,7.0,20.3
4,15.0,20.6


In [42]:
pd.DataFrame({'rfe_ranking': rfe.ranking_}, index=X_train_scaled.columns)


,rfe_ranking
Agriculture,3
Examination,2
Education,1
Catholic,4
Infant.Mortality,1


In [44]:
X_train_scaled.columns[rfe.get_support()]

Index(['Education', 'Infant.Mortality'], dtype='object')